In [1]:
import pandas as pd
import tensorflow as tf


In [30]:
data = pd.read_csv("data/winequality_red.csv")

In [31]:
rep_cols = []
for i in data.columns:
    rep_cols.append(i.replace(" ",""))

In [32]:
data.columns = rep_cols

In [33]:
cols = list(data.columns)
label_col = cols[-1]
input_col = cols[:-1]

In [34]:
feature_columns = []
feature_input = {}
for header in input_col:
    feature_columns.append(tf.feature_column.numeric_column(header))
    feature_input[header] = tf.keras.Input(shape=(1,),dtype=tf.float64, name=header)
    
#label = tf.feature_column.categorical_column_with_vocabulary_list(
#      'y', ['3', '4', '5', '6', '7', '8'])
#label_one_hot = tf.feature_column.indicator_column(label)
#feature_input['y'] = tf.keras.Input(shape=(1,),dtype=tf.string,name=header)

In [35]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size = 0.3)

In [40]:
def data_to_tensor(df,batch_size=32,shuffle=True):
    df = df.copy()
    data_x = df.drop("quality", axis=1)
    ts_data = tf.data.Dataset.from_tensor_slices((dict(data_x),df["quality"]))
    if shuffle:
        ts_data = ts_data.shuffle(buffer_size=len(data))
    ts_data = ts_data.batch(batch_size)
    return ts_data

In [41]:
train_ds = data_to_tensor(train)
test_ds = data_to_tensor(test, shuffle=False)

In [49]:
model = tf.keras.Sequential([
    tf.keras.layers.DenseFeatures(feature_columns=feature_columns),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

In [50]:
model.compile(optimizer='adam',
              loss='mae',
              metrics=['mae'])

In [53]:
model.fit(train_ds, validation_data = test_ds, epochs=10)

Epoch 1/10
35/35 [==============================] - 0s 7ms/step - loss: 0.5546 - mae: 0.5546 - val_loss: 0.0000e+00 - val_mae: 0.0000e+00
Epoch 2/10
35/35 [==============================] - 0s 4ms/step - loss: 0.6236 - mae: 0.6198 - val_loss: 0.4803 - val_mae: 0.4803
Epoch 3/10
35/35 [==============================] - 0s 6ms/step - loss: 0.5444 - mae: 0.5442 - val_loss: 0.5166 - val_mae: 0.5166
Epoch 4/10
35/35 [==============================] - 0s 6ms/step - loss: 0.5371 - mae: 0.5425 - val_loss: 0.4966 - val_mae: 0.4966
Epoch 5/10
35/35 [==============================] - 0s 6ms/step - loss: 0.5515 - mae: 0.5524 - val_loss: 0.5604 - val_mae: 0.5604
Epoch 6/10
35/35 [==============================] - 0s 5ms/step - loss: 0.5488 - mae: 0.5453 - val_loss: 0.4789 - val_mae: 0.4789
Epoch 7/10
35/35 [==============================] - 0s 6ms/step - loss: 0.5404 - mae: 0.5427 - val_loss: 0.5341 - val_mae: 0.5341
Epoch 8/10
35/35 [==============================] - 0s 5ms/step - loss: 0.5497 - m

In [54]:
from witwidget.notebook.visualization import WitConfigBuilder
from witwidget.notebook.visualization import WitWidget


In [55]:
import numpy as np
def df_to_examples(df, columns=None):
    examples = []
    if columns == None:
        columns = df.columns.values.tolist()
    for index, row in df.iterrows():
        example = tf.train.Example()
        for col in columns:
            if df[col].dtype is np.dtype(np.int64):
                example.features.feature[col].int64_list.value.append(int(row[col]))
            elif df[col].dtype is np.dtype(np.float64):
                example.features.feature[col].float_list.value.append(row[col])
            elif row[col] == row[col]:
                example.features.feature[col].bytes_list.value.append(row[col].encode('utf-8'))
        examples.append(example)
    return examples

In [56]:
example = df_to_examples(data)

In [61]:
config_builder = WitConfigBuilder(example).set_estimator_and_feature_spec(
    model, feature_input).set_target_feature('quality').set_model_type('regression')

In [62]:
WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'regression', 'label_vocab': [], 'are_sequence_examples': False, 'inference_ad…